# Bash链
本笔记本展示了如何使用LLMs和bash进程执行简单的文件系统命令。

In [1]:
from langchain_experimental.llm_bash.base import LLMBashChain
from langchain_openai import OpenAI

# 创建一个OpenAI实例，设置温度为0
llm = OpenAI(temperature=0)

# 待生成文本
text = "Please write a bash script that prints 'Hello World' to the console."

# 从OpenAI实例创建LLMBashChain实例
bash_chain = LLMBashChain.from_llm(llm, verbose=True)

# 调用LLMBashChain实例的invoke方法生成bash脚本
bash_chain.invoke(text)



> Entering new LLMBashChain chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
echo "Hello World"
```
Code: ['echo "Hello World"']
Answer: Hello World

> Finished chain.


'Hello World\n'

## 自定义提示符
您还可以自定义使用的提示符。以下是一个示例，提示避免使用“echo”实用程序。

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_experimental.llm_bash.prompt import BashOutputParser

# 定义一个模板字符串，指导用户如何使用bash命令来执行特定任务
_PROMPT_TEMPLATE = """If someone asks you to perform a task, your job is to come up with a series of bash commands that will perform the task. There is no need to put "#!/bin/bash" in your answer. Make sure to reason step by step, using this format:
Question: "copy the files in the directory named 'target' into a new directory at the same level as target called 'myNewDirectory'"
I need to take the following actions:
- List all files in the directory
- Create a new directory
- Copy the files from the first directory into the second directory
ls
mkdir myNewDirectory
cp -r target/* myNewDirectory

Do not use 'echo' when writing the script.

That is the format. Begin!
Question: {question}"""

# 创建一个PromptTemplate实例，用于生成问题提示
PROMPT = PromptTemplate(
    input_variables=["question"],  # 输入变量为"question"
    template=_PROMPT_TEMPLATE,  # 使用上面定义的模板字符串
    output_parser=BashOutputParser(),  # 使用BashOutputParser解析器
)

In [3]:
# 创建一个LLMBashChain对象，并设置相关参数
bash_chain = LLMBashChain.from_llm(llm, prompt=PROMPT, verbose=True)

# 定义一个字符串变量，用于存储要执行的bash脚本内容
text = "Please write a bash script that prints 'Hello World' to the console."

# 调用bash_chain对象的invoke方法，执行bash脚本
bash_chain.invoke(text)



> Entering new LLMBashChain chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
printf "Hello World\n"
```
Code: ['printf "Hello World\\n"']
Answer: Hello World

> Finished chain.


'Hello World\n'

## 持久终端

默认情况下，每次调用链式命令时，它都会在一个单独的子进程中运行。通过使用持久的bash进程进行实例化，可以改变这种行为。

In [4]:
from langchain_experimental.llm_bash.bash import BashProcess  # 导入BashProcess类

persistent_process = BashProcess(persistent=True)  # 创建一个持久的Bash进程对象
bash_chain = LLMBashChain.from_llm(llm, bash_process=persistent_process, verbose=True)  # 使用LLMBashChain类从llm创建一个Bash链，使用持久的Bash进程对象，并设置为详细模式

text = "List the current directory then move up a level."  # 设置要执行的文本命令

bash_chain.invoke(text)  # 调用Bash链的invoke方法执行文本命令



> Entering new LLMBashChain chain...
List the current directory then move up a level.

```bash
ls
cd ..
```
Code: ['ls', 'cd ..']
Answer: cpal.ipynb  llm_bash.ipynb  llm_symbolic_math.ipynb
index.mdx   llm_math.ipynb  pal.ipynb
> Finished chain.


'cpal.ipynb  llm_bash.ipynb  llm_symbolic_math.ipynb\r\nindex.mdx   llm_math.ipynb  pal.ipynb'

In [5]:
# 再次运行相同的命令，并观察调用之间的状态保持
bash_chain.invoke(text)



> Entering new LLMBashChain chain...
List the current directory then move up a level.

```bash
ls
cd ..
```
Code: ['ls', 'cd ..']
Answer: _category_.yml	data_generation.ipynb		   self_check
agents		graph
code_writing	learned_prompt_optimization.ipynb
> Finished chain.


'_category_.yml\tdata_generation.ipynb\t\t   self_check\r\nagents\t\tgraph\r\ncode_writing\tlearned_prompt_optimization.ipynb'